### Limpieza de la base de datos
Proyecto: Análisis tutelar en temas de salud 1992 - 2019

Autor: Juan Sebastian Muñoz

Objetivo: Limpiar la información de las tutelas.

Fecha de creación: Noviembre de 2019

In [88]:
import nltk                     #Para Análisis del texto
from nltk import word_tokenize
from nltk.corpus import stopwords
import numpy as np
import timeit                   #Para calcular tiempos de ejecución del código
import re                       #Para uso de expresiones regulares
import matplotlib.pyplot as plt #Para realizar los gráficos
import requests                 #Para hacer las solicitudes del codigo fuente de las páginas
from bs4 import BeautifulSoup   #Para transformar el código extraído a formato para poder ser usado en Python
import pandas as pd             #Decarga de pandas para uso de bases de datos
plt.rcParams["figure.figsize"] = [18.0, 8.0]

In [89]:
Database_tutelas = pd.read_pickle('DB_Tutelas.pkl')
len(Database_tutelas)

5837

In [90]:
Database_tutelas

,Item,Expediente,Año,Mes,Día,Link,Sentencia,Magistrado_Ponente,Demandante,Demandado,Tema
0,1,2130,1992,08,11,http://www.corteconstitucional.gov.co//relator...,T-484/92,FABIO MORÓN DÍAZ,ALONSO MUÑOZ,.,DERECHO A LA SALUD. DERECHO A LA VIDA. SIDA. ...
1,2,2047,1992,08,11,http://www.corteconstitucional.gov.co//relator...,T-487/92,ALEJANDRO MARTÍNEZ CABALLERO,DANILO MOLINA,.,DERECHO A LA SALUD. LIBERTAD DE LOCOMOCION. L...
2,3,2193,1992,08,13,http://www.corteconstitucional.gov.co//relator...,T-491/92,EDUARDO CIFUENTES MUÑOZ,ROSALBA DUQUE,.,DERECHO A LA SEGURIDAD SOCIAL. DERECHO A LA S...
3,4,2359,1992,08,21,http://www.corteconstitucional.gov.co//relator...,T-499/92,EDUARDO CIFUENTES MUÑOZ,NERY CHIQUIZA,.,DERECHO A LA SALUD. DIGNIDAD HUMANA. CAPRECOM...
4,5,2535,1992,08,28,http://www.corteconstitucional.gov.co//relator...,T-505/92,EDUARDO CIFUENTES MUÑOZ,DIEGO SERNA,.,SERVICIO DE SALUD. DERECHOS DEL ENFERMO. SIDA...
5,6,2972,1992,09,19,http://www.corteconstitucional.gov.co//relator...,T-522/92,ALEJANDRO MARTÍNEZ CABALLERO,BALDOYNO ASPRILLA,.,DERECHOS DE LOS INTERNOS. DERECHO A LA SALUD....
6,7,2610,1992,09,23,http://www.corteconstitucional.gov.co//relator...,T-536/92,SIMON RODRIGUEZ RODRIGUEZ,OLINDA BARRAGAN Y OTRA,.,DERECHO AL AMBIENTE SANO. DERECHO A LA SALUD....
7,8,3115,1992,09,24,http://www.corteconstitucional.gov.co//relator...,T-534/92,CIRO ANGARITA BARON,JORGE A MORENO,.,DERECHO A LA SALUD. SERVICIO MILITAR. DERECHO...
8,9,3479,1992,10,02,http://www.corteconstitucional.gov.co//relator...,T-548/92,CIRO ANGARITA BARON,FELIX ROJAS,.,DERECHO A LA SALUD. DERECHOS DE LOS PACIENTES...
9,10,D-039,1992,10,20,http://www.corteconstitucional.gov.co//relator...,C-560/92,SIMON RODRIGUEZ RODRIGUEZ,ABDON ORTIZ,.,DEC. 1272/90. ARTS. 3 LIT p) Y 7 LIT ll). REF...


In [91]:
#Limpieza de links y formatos especiales dentro de 'tema'
def clean_color(text):
    return re.sub('<span style="color:black;background:#FF8000">.*?</span>', 'salud', text)
Database_tutelas['Tema'] = Database_tutelas['Tema'].apply(clean_color)
def clean_links(text):
    return re.sub('<a.*?a>', '', text)
Database_tutelas['Tema'] = Database_tutelas['Tema'].apply(clean_links)
Database_tutelas['Tema'][5000]

'   Salud, vida digna, integridad, seguridad social.  Se acumulan expedientes por unidad de materia.  Se le endilga a diferentes E.P.S. la vulneración de derechos fundamentales, por no suministrar a los agenciados los servicios e insumos que reclaman para mejorar la calidad de vida, esto es, pañales desechables, silla de ruedas, tratamiento integral para sus patologías y exoneración de copagos y cuotas moderadoras, bajo el argumento de estar excluidos del POS.   El derecho fundamental a la'

In [92]:
#Defino puntuación para eliminarla
puntuacion = '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~–¡¿”“•\r´–¡¿”“•\r´º'
puntuacion

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~–¡¿”“•\r´–¡¿”“•\r´º'

In [93]:
#Defino una función que elimine la puntuación
def no_punt(string):
    transtable = string.maketrans('', '', puntuacion)
    return string.translate(transtable)
Database_tutelas['Tutela_no_punt'] = Database_tutelas['Tema'].apply(no_punt)

In [94]:
print(Database_tutelas['Tema'][5800])

 DERECHO AL GOCE EFECTIVO DE LA MESADA PENSIONAL, ANTE IMPOSIBILIDAD FÍSICA Y/O MENTAL DEL TITULAR DEL DERECHO PARA RECLAMARLA 
 La accionante actúa como agente oficiosa de su cónyuge quien tiene más de 85 años de edad, padece demencia senil, alcoholismo crónico y otras patologías.  En la misma condición actúa en favor de un hijo mayor de edad diagnosticado con el síndrome de Gilles de la Tourette.  El esposo de la actora es beneficiario de una pensión de vejez y esta prestación es la única fuente de ingresos del núcleo familiar. La peticionaria es quien hace los retiros de las mesadas pensionales en la entidad bancaria accionada, a través de una tarjeta débito a nombre del pensionado, la cual se venció y debe ser renovada sólo por el titular de la cuenta de ahorros y a través de su presentación personal.  Este trámite es el que exige el banco y el que la accionante alega como imposible de cumplir, debido a las condiciones de


In [95]:
print(Database_tutelas['Tutela_no_punt'][5800])

 DERECHO AL GOCE EFECTIVO DE LA MESADA PENSIONAL ANTE IMPOSIBILIDAD FÍSICA YO MENTAL DEL TITULAR DEL DERECHO PARA RECLAMARLA 
 La accionante actúa como agente oficiosa de su cónyuge quien tiene más de 85 años de edad padece demencia senil alcoholismo crónico y otras patologías  En la misma condición actúa en favor de un hijo mayor de edad diagnosticado con el síndrome de Gilles de la Tourette  El esposo de la actora es beneficiario de una pensión de vejez y esta prestación es la única fuente de ingresos del núcleo familiar La peticionaria es quien hace los retiros de las mesadas pensionales en la entidad bancaria accionada a través de una tarjeta débito a nombre del pensionado la cual se venció y debe ser renovada sólo por el titular de la cuenta de ahorros y a través de su presentación personal  Este trámite es el que exige el banco y el que la accionante alega como imposible de cumplir debido a las condiciones de


In [96]:
#Paso a minúsculas todas las letras
Database_tutelas['Tutela_no_punt'] = Database_tutelas['Tutela_no_punt'].apply(str.lower)

In [97]:
Database_tutelas['Demandado_no_punt'] = Database_tutelas['Demandado'].apply(no_punt)
Database_tutelas['Demandado_no_punt'] = Database_tutelas['Demandado_no_punt'].apply(str.lower)

In [98]:
#Deja solo al magistrado ponente:
def clean_magistrado(text):
    return re.sub('<.*?   ', '', text)
Database_tutelas['Magistrado_Ponente'] = Database_tutelas['Magistrado_Ponente'].apply(clean_magistrado)
Database_tutelas['Magistrado_Ponente'][0]

'FABIO MORÓN DÍAZ                   '

# Quitar el vs de demandado....

In [ ]:
def clean_demandado(text):
    return re.sub('<.*?   ', '', text)

In [99]:
Database_tutelas['Año'] = pd.to_numeric(Database_tutelas['Año'], errors = "coerce")
Database_tutelas['Mes'] = pd.to_numeric(Database_tutelas['Mes'], errors = "coerce")
Database_tutelas['Día'] = pd.to_numeric(Database_tutelas['Día'], errors = "coerce")

In [100]:
# Añadir la columna de fecha en formato datetime

Database_tutelas['Fecha'] = pd.to_datetime(Database_tutelas.Año * 10000 +\
                                           Database_tutelas.Mes * 100 + Database_tutelas.Día, format='%Y%m%d')

In [101]:
Database_tutelas

,Item,Expediente,Año,Mes,Día,Link,Sentencia,Magistrado_Ponente,Demandante,Demandado,Tema,Tutela_no_punt,Demandado_no_punt,Fecha
0,1,2130,1992,8,11,http://www.corteconstitucional.gov.co//relator...,T-484/92,FABIO MORÓN DÍAZ,ALONSO MUÑOZ,.,DERECHO A LA SALUD. DERECHO A LA VIDA. SIDA. ...,derecho a la salud derecho a la vida sida iss...,,1992-08-11
1,2,2047,1992,8,11,http://www.corteconstitucional.gov.co//relator...,T-487/92,ALEJANDRO MARTÍNEZ CABALLERO,DANILO MOLINA,.,DERECHO A LA SALUD. LIBERTAD DE LOCOMOCION. L...,derecho a la salud libertad de locomocion lib...,,1992-08-11
2,3,2193,1992,8,13,http://www.corteconstitucional.gov.co//relator...,T-491/92,EDUARDO CIFUENTES MUÑOZ,ROSALBA DUQUE,.,DERECHO A LA SEGURIDAD SOCIAL. DERECHO A LA S...,derecho a la seguridad social derecho a la sa...,,1992-08-13
3,4,2359,1992,8,21,http://www.corteconstitucional.gov.co//relator...,T-499/92,EDUARDO CIFUENTES MUÑOZ,NERY CHIQUIZA,.,DERECHO A LA SALUD. DIGNIDAD HUMANA. CAPRECOM...,derecho a la salud dignidad humana caprecom c...,,1992-08-21
4,5,2535,1992,8,28,http://www.corteconstitucional.gov.co//relator...,T-505/92,EDUARDO CIFUENTES MUÑOZ,DIEGO SERNA,.,SERVICIO DE SALUD. DERECHOS DEL ENFERMO. SIDA...,servicio de salud derechos del enfermo sida c...,,1992-08-28
5,6,2972,1992,9,19,http://www.corteconstitucional.gov.co//relator...,T-522/92,ALEJANDRO MARTÍNEZ CABALLERO,BALDOYNO ASPRILLA,.,DERECHOS DE LOS INTERNOS. DERECHO A LA SALUD....,derechos de los internos derecho a la salud c...,,1992-09-19
6,7,2610,1992,9,23,http://www.corteconstitucional.gov.co//relator...,T-536/92,SIMON RODRIGUEZ RODRIGUEZ,OLINDA BARRAGAN Y OTRA,.,DERECHO AL AMBIENTE SANO. DERECHO A LA SALUD....,derecho al ambiente sano derecho a la salud m...,,1992-09-23
7,8,3115,1992,9,24,http://www.corteconstitucional.gov.co//relator...,T-534/92,CIRO ANGARITA BARON,JORGE A MORENO,.,DERECHO A LA SALUD. SERVICIO MILITAR. DERECHO...,derecho a la salud servicio militar derecho a...,,1992-09-24
8,9,3479,1992,10,2,http://www.corteconstitucional.gov.co//relator...,T-548/92,CIRO ANGARITA BARON,FELIX ROJAS,.,DERECHO A LA SALUD. DERECHOS DE LOS PACIENTES...,derecho a la salud derechos de los pacientes ...,,1992-10-02
9,10,D-039,1992,10,20,http://www.corteconstitucional.gov.co//relator...,C-560/92,SIMON RODRIGUEZ RODRIGUEZ,ABDON ORTIZ,.,DEC. 1272/90. ARTS. 3 LIT p) Y 7 LIT ll). REF...,dec 127290 arts 3 lit p y 7 lit ll reforma de...,,1992-10-20


In [102]:
#Quita el doble espacio
def clean_spaces(text):
    return re.sub(' {2,}', '', text)
Database_tutelas['Magistrado_Ponente'] = Database_tutelas['Magistrado_Ponente'].apply(clean_spaces)

In [103]:
#Quita puntos, especialmente para que palabras como E.P.S. no vayan a quedar en tres palabras diferentes
def clean_dot(text):
    return re.sub('\.', '', text)
Database_tutelas['Demandado'] = Database_tutelas['Demandado'].apply(clean_dot)
def clean_comma(text):
    return re.sub(',', ' ', text)
Database_tutelas['Demandado'] = Database_tutelas['Demandado'].apply(clean_comma)

In [104]:
Database_tutelas['Demandado'][600:1000]

600                                                     
601                                                     
602     ESE HOSPITAL LA MARIA DE MEDELLIN Y EL SISBEN...
603                                                     
604                             ALVARO URREA BOTERO VS  
605     INSTITUTO DE SEGURO SOCIAL SECCIONAL CUNDINAM...
606                                                     
607     INSTITUTO DE SUGUROS SOCIALES Y OTRO (VALLE) ...
608                                   SALUDCOOP EPS VS  
609                               SALUD TOTAL - EPS VS  
610           INSTITUTO DE SEGUROS SOCIALES (VALLE) VS  
611                                POLICIA NACIONAL VS  
612                             ACERIAS PAZ DEL RIO VS  
613                                   CAFESALUD EPS VS  
614                                   CAFESALUD EPS VS  
615     INSTITUTO DE SEGUROS SOCIALES DE BARRANQUILLA...
616                   INSTITUTO DE SEGUROS SOCIALES VS  
617                            

In [105]:
#Revisar que no hayan duplicados
Database_tutelas.drop_duplicates(subset = ('Sentencia'), inplace = True)
len(Database_tutelas)

5834

5837 - 3834 = 3

Por lo tanto, hay 3 duplicados

In [106]:
# Organizo por fecha

Database_tutelas.sort_values('Fecha', inplace = True)

In [107]:
# Reinicio los índices

Database_tutelas.reset_index(inplace = True, drop = True)
Database_tutelas

,Item,Expediente,Año,Mes,Día,Link,Sentencia,Magistrado_Ponente,Demandante,Demandado,Tema,Tutela_no_punt,Demandado_no_punt,Fecha
0,1,2130,1992,8,11,http://www.corteconstitucional.gov.co//relator...,T-484/92,FABIO MORÓN DÍAZ,ALONSO MUÑOZ,,DERECHO A LA SALUD. DERECHO A LA VIDA. SIDA. ...,derecho a la salud derecho a la vida sida iss...,,1992-08-11
1,2,2047,1992,8,11,http://www.corteconstitucional.gov.co//relator...,T-487/92,ALEJANDRO MARTÍNEZ CABALLERO,DANILO MOLINA,,DERECHO A LA SALUD. LIBERTAD DE LOCOMOCION. L...,derecho a la salud libertad de locomocion lib...,,1992-08-11
2,3,2193,1992,8,13,http://www.corteconstitucional.gov.co//relator...,T-491/92,EDUARDO CIFUENTES MUÑOZ,ROSALBA DUQUE,,DERECHO A LA SEGURIDAD SOCIAL. DERECHO A LA S...,derecho a la seguridad social derecho a la sa...,,1992-08-13
3,4,2359,1992,8,21,http://www.corteconstitucional.gov.co//relator...,T-499/92,EDUARDO CIFUENTES MUÑOZ,NERY CHIQUIZA,,DERECHO A LA SALUD. DIGNIDAD HUMANA. CAPRECOM...,derecho a la salud dignidad humana caprecom c...,,1992-08-21
4,5,2535,1992,8,28,http://www.corteconstitucional.gov.co//relator...,T-505/92,EDUARDO CIFUENTES MUÑOZ,DIEGO SERNA,,SERVICIO DE SALUD. DERECHOS DEL ENFERMO. SIDA...,servicio de salud derechos del enfermo sida c...,,1992-08-28
5,6,2972,1992,9,19,http://www.corteconstitucional.gov.co//relator...,T-522/92,ALEJANDRO MARTÍNEZ CABALLERO,BALDOYNO ASPRILLA,,DERECHOS DE LOS INTERNOS. DERECHO A LA SALUD....,derechos de los internos derecho a la salud c...,,1992-09-19
6,7,2610,1992,9,23,http://www.corteconstitucional.gov.co//relator...,T-536/92,SIMON RODRIGUEZ RODRIGUEZ,OLINDA BARRAGAN Y OTRA,,DERECHO AL AMBIENTE SANO. DERECHO A LA SALUD....,derecho al ambiente sano derecho a la salud m...,,1992-09-23
7,8,3115,1992,9,24,http://www.corteconstitucional.gov.co//relator...,T-534/92,CIRO ANGARITA BARON,JORGE A MORENO,,DERECHO A LA SALUD. SERVICIO MILITAR. DERECHO...,derecho a la salud servicio militar derecho a...,,1992-09-24
8,9,3479,1992,10,2,http://www.corteconstitucional.gov.co//relator...,T-548/92,CIRO ANGARITA BARON,FELIX ROJAS,,DERECHO A LA SALUD. DERECHOS DE LOS PACIENTES...,derecho a la salud derechos de los pacientes ...,,1992-10-02
9,10,D-039,1992,10,20,http://www.corteconstitucional.gov.co//relator...,C-560/92,SIMON RODRIGUEZ RODRIGUEZ,ABDON ORTIZ,,DEC. 1272/90. ARTS. 3 LIT p) Y 7 LIT ll). REF...,dec 127290 arts 3 lit p y 7 lit ll reforma de...,,1992-10-20


In [108]:
#Creo una lista de tokens
Database_tutelas['Tutela_tkn'] = Database_tutelas['Tutela_no_punt'].apply(word_tokenize)
Database_tutelas['Demandado_tkn'] = Database_tutelas['Demandado_no_punt'].apply(word_tokenize)

In [109]:
# Adquiero las stopwords en español y las elimino
stopwords_esp = stopwords.words('spanish')
def no_stopwords(tokens):
    return [p for p in tokens if p not in stopwords_esp]

In [110]:
#Agrego una columna para la tutela y el demandado sin stopwords
Database_tutelas['Tutela_nsw'] = Database_tutelas['Tutela_tkn'].apply(no_stopwords)
Database_tutelas['Demandado_nsw'] = Database_tutelas['Demandado_tkn'].apply(no_stopwords)

In [111]:
#Paso a texto los tokens de los demandados y de las tutelas
Database_tutelas['Tutela_text'] = Database_tutelas['Tutela_nsw'].apply(nltk.Text)
Database_tutelas['Demandado_text'] = Database_tutelas['Demandado_nsw'].apply(nltk.Text)

In [112]:
Database_tutelas.loc[5000:5100]

,Item,Expediente,Año,Mes,Día,Link,Sentencia,Magistrado_Ponente,Demandante,Demandado,Tema,Tutela_no_punt,Demandado_no_punt,Fecha,Tutela_tkn,Demandado_tkn,Tutela_nsw,Demandado_nsw,Tutela_text,Demandado_text
5000,5004,T-4032900,2014,1,27,/relatoria/2014/T-028-14.htm,T-028/14,MARÍA VICTORIA CALLE CORREA,YOSIRA COROMOTO BERMUDEZ,AGUAS DE LA PENINSULA SA ESP,Se le endilga a la empresa Aguas de la Pení...,se le endilga a la empresa aguas de la pení...,aguas de la peninsula sa esp,2014-01-27,"[se, le, endilga, a, la, empresa, aguas, de, l...","[aguas, de, la, peninsula, sa, esp]","[endilga, empresa, aguas, península, sa, munic...","[aguas, peninsula, sa, esp]","(endilga, empresa, aguas, península, sa, munic...","(aguas, peninsula, sa, esp)"
5001,5006,LAT-411,2014,1,29,/relatoria/2014/C-036-14.htm,C-036/14,LUIS ERNESTO VARGAS SILVA,REVISION CONSTITUCIONAL DE LA LEY 1623 DE 2013...,,Revisión Constitucional de la Ley 1623 de 2...,revisión constitucional de la ley 1623 de 2...,,2014-01-29,"[revisión, constitucional, de, la, ley, 1623, ...",[],"[revisión, constitucional, ley, 1623, 2013, me...",[],"(revisión, constitucional, ley, 1623, 2013, me...",()
5002,5007,T-4042165,2014,1,31,/relatoria/2014/T-042-14.htm,T-042/14,LUIS ERNESTO VARGAS SILVA,LEVIS MARINA POTE DE LAS SALAS,SECRETARIA DE EDUCACION DEPARTAMENTAL DE BOLIVAR,Aduce la peticionaria que sus derechos fund...,aduce la peticionaria que sus derechos fund...,secretaria de educacion departamental de bolivar,2014-01-31,"[aduce, la, peticionaria, que, sus, derechos, ...","[secretaria, de, educacion, departamental, de,...","[aduce, peticionaria, derechos, fundamentales,...","[secretaria, educacion, departamental, bolivar]","(aduce, peticionaria, derechos, fundamentales,...","(secretaria, educacion, departamental, bolivar)"
5003,5008,T-4052900 Y OTRO ACUMULADOS,2014,2,3,/relatoria/2014/T-048-14.htm,T-048/14,GABRIEL EDUARDO MENDOZA MARTELO,RUTH EMENITH DUARTE BAUTISTA Y OTRO,NUEVA EPS Y OTRA,"Salud, vida, dignidad humana. Se acumulan ...",salud vida dignidad humana se acumulan exp...,nueva eps y otra,2014-02-03,"[salud, vida, dignidad, humana, se, acumulan, ...","[nueva, eps, y, otra]","[salud, vida, dignidad, humana, acumulan, expe...","[nueva, eps]","(salud, vida, dignidad, humana, acumulan, expe...","(nueva, eps)"
5004,5009,T-4061327,2014,2,3,/relatoria/2014/T-052-14.htm,T-052/14,ALBERTO ROJAS RÍOS,JUAN MONTOYA RAMIREZ,UNIDAD ADMINISTRATIVA ESPECIAL DE GESTION PEN...,El accionante solicita que se le protejan s...,el accionante solicita que se le protejan s...,unidad administrativa especial de gestion pen...,2014-02-03,"[el, accionante, solicita, que, se, le, protej...","[unidad, administrativa, especial, de, gestion...","[accionante, solicita, protejan, derechos, fun...","[unidad, administrativa, especial, gestion, pe...","(accionante, solicita, protejan, derechos, fun...","(unidad, administrativa, especial, gestion, pe..."
5005,5010,T-4053884 Y OTROS ACUMULADOS,2014,2,3,/relatoria/2014/T-054-14.htm,T-054/14,ALBERTO ROJAS RÍOS,YULIS ELENA NAVARRO JARAMILLO Y OTROS,EPS CAPRECOM Y OTRAS,"Salud, vida en condiciones dignas. Se acum...",salud vida en condiciones dignas se acumul...,eps caprecom y otras,2014-02-03,"[salud, vida, en, condiciones, dignas, se, acu...","[eps, caprecom, y, otras]","[salud, vida, condiciones, dignas, acumulan, e...","[eps, caprecom]","(salud, vida, condiciones, dignas, acumulan, e...","(eps, caprecom)"
5006,5011,T-4054261 Y OTRO ACUMULADOS,2014,2,3,/relatoria/2014/T-061-14.htm,T-061/14,NILSON PINILLA PINILLA,BLANCA ALICIA CARVAJAL DIAZ Y OTROS,SALUD TOTAL EPS Y OTROS,"Salud, integridad física, vida digna. Se a...",salud integridad física vida digna se acum...,salud total eps y otros,2014-02-03,"[salud, integridad, física, vida, digna, se, a...","[salud, total, eps, y, otros]","[salud, integridad, física, vida, digna, acumu...","[salud, total, eps]","(salud, integridad, física, vida, digna, acumu...","(salud, total, eps)"
5007,5012,T-4061267,2014,2,3,/relatoria/2014

In [113]:
Database_tutelas['Respuesta'] =[x for x in range (len(Database_tutelas))]

In [116]:
Database_tutelas

,Item,Expediente,Año,Mes,Día,Link,Sentencia,Magistrado_Ponente,Demandante,Demandado,...,Tutela_no_punt,Demandado_no_punt,Fecha,Tutela_tkn,Demandado_tkn,Tutela_nsw,Demandado_nsw,Tutela_text,Demandado_text,Respuesta
0,1,2130,1992,8,11,http://www.corteconstitucional.gov.co//relator...,T-484/92,FABIO MORÓN DÍAZ,ALONSO MUÑOZ,,...,derecho a la salud derecho a la vida sida iss...,,1992-08-11,"[derecho, a, la, salud, derecho, a, la, vida, ...",[],"[derecho, salud, derecho, vida, sida, iss, con...",[],"(derecho, salud, derecho, vida, sida, iss, con...",(),1
1,2,2047,1992,8,11,http://www.corteconstitucional.gov.co//relator...,T-487/92,ALEJANDRO MARTÍNEZ CABALLERO,DANILO MOLINA,,...,derecho a la salud libertad de locomocion lib...,,1992-08-11,"[derecho, a, la, salud, libertad, de, locomoci...",[],"[derecho, salud, libertad, locomocion, liberta...",[],"(derecho, salud, libertad, locomocion, liberta...",(),1
2,3,2193,1992,8,13,http://www.corteconstitucional.gov.co//relator...,T-491/92,EDUARDO CIFUENTES MUÑOZ,ROSALBA DUQUE,,...,derecho a la seguridad social derecho a la sa...,,1992-08-13,"[derecho, a, la, seguridad, social, derecho, a...",[],"[derecho, seguridad, social, derecho, salud, p...",[],"(derecho, seguridad, social, derecho, salud, p...",(),1
3,4,2359,1992,8,21,http://www.corteconstitucional.gov.co//relator...,T-499/92,EDUARDO CIFUENTES MUÑOZ,NERY CHIQUIZA,,...,derecho a la salud dignidad humana caprecom c...,,1992-08-21,"[derecho, a, la, salud, dignidad, humana, capr...",[],"[derecho, salud, dignidad, humana, caprecom, c...",[],"(derecho, salud, dignidad, humana, caprecom, c...",(),1
4,5,2535,1992,8,28,http://www.corteconstitucional.gov.co//relator...,T-505/92,EDUARDO CIFUENTES MUÑOZ,DIEGO SERNA,,...,servicio de salud derechos del enfermo sida c...,,1992-08-28,"[servicio, de, salud, derechos, del, enfermo, ...",[],"[servicio, salud, derechos, enfermo, sida, con...",[],"(servicio, salud, derechos, enfermo, sida, con...",(),1
5,6,2972,1992,9,19,http://www.corteconstitucional.gov.co//relator...,T-522/92,ALEJANDRO MARTÍNEZ CABALLERO,BALDOYNO ASPRILLA,,...,derechos de los internos derecho a la salud c...,,1992-09-19,"[derechos, de, los, internos, derecho, a, la, ...",[],"[derechos, internos, derecho, salud, concedida]",[],"(derechos, internos, derecho, salud, concedida)",(),1
6,7,2610,1992,9,23,http://www.corteconstitucional.gov.co//relator...,T-536/92,SIMON RODRIGUEZ RODRIGUEZ,OLINDA BARRAGAN Y OTRA,,...,derecho al ambiente sano derecho a la salud m...,,1992-09-23,"[derecho, al, ambiente, sano, derecho, a, la, ...",[],"[derecho, ambiente, sano, derecho, salud, medi...",[],"(derecho, ambiente, sano, derecho, salud, medi...",(),1
7,8,3115,1992,9,24,http://www.corteconstitucional.gov.co//relator...,T-534/92,CIRO ANGARITA BARON,JORGE A MORENO,,...,derecho a la salud servicio militar derecho a...,,1992-09-24,"[derecho, a, la, salud, servicio, militar, der...",[],"[derecho, salud, servicio, militar, derecho, s...",[],"(derecho, salud, servicio, militar, derecho, s...",(),1
8,9,3479,1992,10,2,http://www.corteconstitucional.gov.co//relator...,T-548/92,CIRO ANGARITA BARON,FELIX ROJAS,,...,derecho a la salud derechos de los pacientes ...,,1992-10-02,"[derecho, a, la, salud, derechos, de, los, pac...",[],"[derecho, salud, derechos, pacientes, iss, neg...",[],"(derecho, salud, derechos, pacientes, iss, neg...",(),-1
9,10,D-039,1992,10,20,http://www.corteconstitucional.gov.co//relator...,C-560/92,SIMON RODRIGUEZ RODRIGUEZ,ABDON ORTIZ,,...,dec 127290 arts 3 lit p y 7 lit ll reforma de...,,1992-10-20,"[dec, 127290, arts, 3, lit, p, y, 7, lit, ll, ...",[],"[dec, 127290, arts, 3, lit, p, 7, lit, ll, ref...",[],"(dec, 127290, arts, 3, lit, p, 7, lit, ll, ref...",(),12


In [115]:
# Se intenta identificar si la tutela fue Concedida, concedida parcialmente, negada, etc.
# Negada = -1
# Concedida Parcialmente = 0.5
# Concedida, AMPARADA = 1
# Hecho superado = 0
# Improcedencia = 2
# Denegar = 3
# Tutelan los derechos = 4
# EXEQUIBILIDAD CONDICIONADA = 5
# Exequibilidad = 6
# Inhibe = 7
# DAÑO CONSUMADO = 8
# CARENCIA ACTUAL DE OBJETO = 9
# CONFIRMA PARCIALMENTE =10
# Se reitera = 11
# Inexequible = 12
# Someterse a lo resuleto = 13
# No procede = 14

for i in range(len(Database_tutelas)):
    if str(Database_tutelas['Tema'][i]).find(' NEGADA.<br/>') >= 1:
        Database_tutelas.loc[[i], 'Respuesta'] = '-1'
    elif str(Database_tutelas['Tema'][i]).find(' NEGADA') >= 1:
        Database_tutelas.loc[[i], 'Respuesta'] = -1
    elif str(Database_tutelas['Tema'][i]).find('<br/>NEGADA') >= 1:
        Database_tutelas.loc[[i], 'Respuesta'] = -1
    elif str(Database_tutelas['Tema'][i]).find('<br/>NEGADA.') >= 1:
        Database_tutelas.loc[[i], 'Respuesta'] = -1
    elif str(Database_tutelas['Tema'][i]).find('\r<br/>NEGADA\r<br/>') >= 1:
        Database_tutelas.loc[[i], 'Respuesta'] = -1
    elif str(Database_tutelas['Tema'][i]).find(' NEGARON ') >= 1:
        Database_tutelas.loc[[i], 'Respuesta'] = -1
    elif str(Database_tutelas['Tema'][i]).find(' NEGÓ ') >= 1:
        Database_tutelas.loc[[i], 'Respuesta'] = -1
    elif str(Database_tutelas['Tema'][i]).find(' NEGAR ') >= 1:
        Database_tutelas.loc[[i], 'Respuesta'] = -1
    elif str(Database_tutelas['Tema'][i]).find('NIEGA') >= 1:
        Database_tutelas.loc[[i], 'Respuesta'] = -1
    elif str(Database_tutelas['Tema'][i]).find(' NIEG') >= 1:
        Database_tutelas.loc[[i], 'Respuesta'] = -1
    elif str(Database_tutelas['Tema'][i]).find(' NEGAD.') >= 1:
        Database_tutelas.loc[[i], 'Respuesta'] = -1
    elif str(Database_tutelas['Tema'][i]).find('HECHO SUPERADO') >= 1:
        Database_tutelas.loc[[i], 'Respuesta'] = 0
    elif str(Database_tutelas['Tema'][i]).find('CONCEDIDA PARCIALMENTE.<br/>') >= 1:
        Database_tutelas.loc[[i], 'Respuesta'] = 0.5
    elif str(Database_tutelas['Tema'][i]).find('CONCEDIDA.<br/>') >= 1:
        Database_tutelas.loc[[i], 'Respuesta'] = 1
    elif str(Database_tutelas['Tema'][i]).find(' CONCED') >= 1:
        Database_tutelas.loc[[i], 'Respuesta'] = 1
    elif str(Database_tutelas['Tema'][i]).find('e conced') >= 1:
        Database_tutelas.loc[[i], 'Respuesta'] = 1
    elif str(Database_tutelas['Tema'][i]).find('AMPAR') >= 1:
        Database_tutelas.loc[[i], 'Respuesta'] = 1
    elif str(Database_tutelas['Tema'][i]).find('TUTELÓ') >= 1:
        Database_tutelas.loc[[i], 'Respuesta'] = 1
    elif str(Database_tutelas['Tema'][i]).find('Se TUTELA') >= 1:
        Database_tutelas.loc[[i], 'Respuesta'] = 1
    elif str(Database_tutelas['Tema'][i]).find('TUTELAR') >= 1:
        Database_tutelas.loc[[i], 'Respuesta'] = 1
    elif str(Database_tutelas['Tema'][i]).find('OTORGA') >= 1:
        Database_tutelas.loc[[i], 'Respuesta'] = 1
    elif str(Database_tutelas['Tema'][i]).find('IMPROCEDENCIA') >= 1:
        Database_tutelas.loc[[i], 'Respuesta'] = 2
    elif str(Database_tutelas['Tema'][i]).find('IMPROCEDENTE') >= 1:
        Database_tutelas.loc[[i], 'Respuesta'] = 2
    elif str(Database_tutelas['Tema'][i]).find('DENIEG.') >= 1:
        Database_tutelas.loc[[i], 'Respuesta'] = 3
    elif str(Database_tutelas['Tema'][i]).find('DENIEGA') >= 1:
        Database_tutelas.loc[[i], 'Respuesta'] = 3
    elif str(Database_tutelas['Tema'][i]).find(' DENEG') >= 1:
        Database_tutelas.loc[[i], 'Respuesta'] = 3
    elif str(Database_tutelas['Tema'][i]).find(' TUTELAN') >= 1:
        Database_tutelas.loc[[i], 'Respuesta'] = 4
    elif str(Database_tutelas['Tema'][i]).find(' EXEQUIBILIDAD CONDICIONADA') >= 1:
        Database_tutelas.loc[[i], 'Respuesta'] = 5
    elif str(Database_tutelas['Tema'][i]).find(' EXEQUIBLE') >= 1:
        Database_tutelas.loc[[i], 'Respuesta'] = 6
    elif str(Database_tutelas['Tema'][i]).find(' EXEQUIBLE.') >= 1:
        Database_tutelas.loc[[i], 'Respuesta'] = 6
    elif str(Database_tutelas['Tema'][i]).find(' EXEQUIBILIDAD') >= 1:
        Database_tutelas.loc[[i], 'Respuesta'] = 6
    elif str(Database_tutelas['Tema'][i]).find('INHIBE') >= 1:
        Database_tutelas.loc[[i], 'Respuesta'] = 7
    elif str(Database_tutelas['Tema'][i]).find('INHIBIDA') >= 1:
        Database_tutelas.loc[[i], 'Respuesta'] = 7
    elif str(Database_tutelas['Tema'][i]).find('INHIBID.') >= 1:
        Database_tutelas.loc[[i], 'Respuesta'] = 7
    elif str(Database_tutelas['Tema'][i]).find('DAÑO CONSUMADO') >= 1:
        Database_tutelas.loc[[i], 'Respuesta'] = 8
    elif str(Database_tutelas['Tema'][i]).find('CARENCIA ACTUAL DE OBJETO') >= 1:
        Database_tutelas.loc[[i], 'Respuesta'] = 9
    elif str(Database_tutelas['Tema'][i]).find('carencia actual de objeto') >= 1:
        Database_tutelas.loc[[i], 'Respuesta'] = 9
    elif str(Database_tutelas['Tema'][i]).find('CONFIRMA PARCIALMENTE') >= 1:
        Database_tutelas.loc[[i], 'Respuesta'] = 10
    elif str(Database_tutelas['Tema'][i]).find('Se reitera') >= 1:
        Database_tutelas.loc[[i], 'Respuesta'] = 11
    elif str(Database_tutelas['Tema'][i]).find('INEXEQUIBLE') >= 1:
        Database_tutelas.loc[[i], 'Respuesta'] = 12
    elif str(Database_tutelas['Tema'][i]).find('ESTARSE A LO RESUELTO') >= 1:
        Database_tutelas.loc[[i], 'Respuesta'] = 13
    elif str(Database_tutelas['Tema'][i]).find('Se declara la improcedencia') >= 1:
        Database_tutelas.loc[[i], 'Respuesta'] = 14
    elif str(Database_tutelas['Tema'][i]).find('es improcedente') >= 1:
        Database_tutelas.loc[[i], 'Respuesta'] = 14
    elif str(Database_tutelas['Tema'][i]).find('TUTELA') >= 1:
        Database_tutelas.loc[[i], 'Respuesta'] = 1
    
    else:
        Database_tutelas.loc[[i], 'Respuesta'] = 'NA'

In [83]:
Database_tutelas.loc[54]

Item                                                                 55
Expediente                                                        40965
Año                                                                1994
Mes                                                                  09
Día                                                                  30
Link                  http://www.corteconstitucional.gov.co//relator...
Sentencia                                                      T-431/94
Magistrado_Ponente                  JOSÉ GREGORIO HERNÁNDEZ GALINDO    
Demandante                                         MARIA JOSEFA SARRIA 
Demandado                                                           .  
Tema                   DER. AL AMBIENTE SANO. DER. A LA SALUD. INEFI...
Respuesta                                                             1
Name: 54, dtype: object

In [84]:
for i in range (len(Database_tutelas)):
    if Database_tutelas['Respuesta'][i] == 'NA':
        print(Database_tutelas['Item'][i] + '-' + Database_tutelas['Respuesta'][i])

53-NA
63-NA
141-NA
152-NA
205-NA
228-NA
229-NA
412-NA
417-NA
418-NA
479-NA
553-NA
673-NA
757-NA
768-NA
790-NA
851-NA
874-NA
885-NA
886-NA
887-NA
888-NA
899-NA
924-NA
926-NA
934-NA
940-NA
943-NA
961-NA
1001-NA
1002-NA
1013-NA
1015-NA
1020-NA
1027-NA
1033-NA
1034-NA
1036-NA
1040-NA
1056-NA
1058-NA
1059-NA
1064-NA
1068-NA
1077-NA
1078-NA
1111-NA
1135-NA
1138-NA
1180-NA
1186-NA
1187-NA
1199-NA
1204-NA
1208-NA
1227-NA
1228-NA
1229-NA
1231-NA
1232-NA
1238-NA
1239-NA
1240-NA
1243-NA
1251-NA
1254-NA
1255-NA
1261-NA
1264-NA
1281-NA
1284-NA
1285-NA
1290-NA
1296-NA
1297-NA
1304-NA
1305-NA
1306-NA
1316-NA
1319-NA
1331-NA
1338-NA
1339-NA
1346-NA
1353-NA
1354-NA
1358-NA
1362-NA
1367-NA
1374-NA
1391-NA
1401-NA
1409-NA
1412-NA
1417-NA
1424-NA
1426-NA
1432-NA
1437-NA
1439-NA
1441-NA
1445-NA
1449-NA
1452-NA
1455-NA
1467-NA
1473-NA
1474-NA
1475-NA
1476-NA
1484-NA
1485-NA
1487-NA
1488-NA
1490-NA
1491-NA
1493-NA
1494-NA
1497-NA
1499-NA
1502-NA
1506-NA
1510-NA
1516-NA
1518-NA
1524-NA
1531-NA
1533-NA
1534-NA

4176-NA
4177-NA
4178-NA
4179-NA
4180-NA
4181-NA
4182-NA
4183-NA
4185-NA
4186-NA
4187-NA
4188-NA
4189-NA
4190-NA
4191-NA
4192-NA
4193-NA
4194-NA
4195-NA
4196-NA
4197-NA
4198-NA
4199-NA
4201-NA
4203-NA
4204-NA
4205-NA
4206-NA
4207-NA
4208-NA
4209-NA
4210-NA
4211-NA
4212-NA
4213-NA
4214-NA
4216-NA
4217-NA
4218-NA
4219-NA
4220-NA
4221-NA
4223-NA
4224-NA
4225-NA
4226-NA
4227-NA
4228-NA
4229-NA
4230-NA
4231-NA
4232-NA
4233-NA
4234-NA
4235-NA
4236-NA
4237-NA
4238-NA
4239-NA
4240-NA
4242-NA
4243-NA
4245-NA
4246-NA
4247-NA
4248-NA
4249-NA
4250-NA
4251-NA
4252-NA
4253-NA
4254-NA
4255-NA
4256-NA
4257-NA
4258-NA
4259-NA
4260-NA
4261-NA
4262-NA
4263-NA
4264-NA
4265-NA
4266-NA
4269-NA
4270-NA
4271-NA
4272-NA
4273-NA
4274-NA
4275-NA
4276-NA
4277-NA
4278-NA
4279-NA
4280-NA
4281-NA
4282-NA
4283-NA
4284-NA
4285-NA
4286-NA
4287-NA
4288-NA
4289-NA
4290-NA
4292-NA
4293-NA
4294-NA
4296-NA
4297-NA
4298-NA
4299-NA
4300-NA
4304-NA
4305-NA
4306-NA
4307-NA
4308-NA
4309-NA
4310-NA
4311-NA
4312-NA
4314-NA
4315-NA


In [117]:
Database_tutelas.to_pickle('DbTutelas.pkl')

In [118]:
Database_tutelas

,Item,Expediente,Año,Mes,Día,Link,Sentencia,Magistrado_Ponente,Demandante,Demandado,...,Tutela_no_punt,Demandado_no_punt,Fecha,Tutela_tkn,Demandado_tkn,Tutela_nsw,Demandado_nsw,Tutela_text,Demandado_text,Respuesta
0,1,2130,1992,8,11,http://www.corteconstitucional.gov.co//relator...,T-484/92,FABIO MORÓN DÍAZ,ALONSO MUÑOZ,,...,derecho a la salud derecho a la vida sida iss...,,1992-08-11,"[derecho, a, la, salud, derecho, a, la, vida, ...",[],"[derecho, salud, derecho, vida, sida, iss, con...",[],"(derecho, salud, derecho, vida, sida, iss, con...",(),1
1,2,2047,1992,8,11,http://www.corteconstitucional.gov.co//relator...,T-487/92,ALEJANDRO MARTÍNEZ CABALLERO,DANILO MOLINA,,...,derecho a la salud libertad de locomocion lib...,,1992-08-11,"[derecho, a, la, salud, libertad, de, locomoci...",[],"[derecho, salud, libertad, locomocion, liberta...",[],"(derecho, salud, libertad, locomocion, liberta...",(),1
2,3,2193,1992,8,13,http://www.corteconstitucional.gov.co//relator...,T-491/92,EDUARDO CIFUENTES MUÑOZ,ROSALBA DUQUE,,...,derecho a la seguridad social derecho a la sa...,,1992-08-13,"[derecho, a, la, seguridad, social, derecho, a...",[],"[derecho, seguridad, social, derecho, salud, p...",[],"(derecho, seguridad, social, derecho, salud, p...",(),1
3,4,2359,1992,8,21,http://www.corteconstitucional.gov.co//relator...,T-499/92,EDUARDO CIFUENTES MUÑOZ,NERY CHIQUIZA,,...,derecho a la salud dignidad humana caprecom c...,,1992-08-21,"[derecho, a, la, salud, dignidad, humana, capr...",[],"[derecho, salud, dignidad, humana, caprecom, c...",[],"(derecho, salud, dignidad, humana, caprecom, c...",(),1
4,5,2535,1992,8,28,http://www.corteconstitucional.gov.co//relator...,T-505/92,EDUARDO CIFUENTES MUÑOZ,DIEGO SERNA,,...,servicio de salud derechos del enfermo sida c...,,1992-08-28,"[servicio, de, salud, derechos, del, enfermo, ...",[],"[servicio, salud, derechos, enfermo, sida, con...",[],"(servicio, salud, derechos, enfermo, sida, con...",(),1
5,6,2972,1992,9,19,http://www.corteconstitucional.gov.co//relator...,T-522/92,ALEJANDRO MARTÍNEZ CABALLERO,BALDOYNO ASPRILLA,,...,derechos de los internos derecho a la salud c...,,1992-09-19,"[derechos, de, los, internos, derecho, a, la, ...",[],"[derechos, internos, derecho, salud, concedida]",[],"(derechos, internos, derecho, salud, concedida)",(),1
6,7,2610,1992,9,23,http://www.corteconstitucional.gov.co//relator...,T-536/92,SIMON RODRIGUEZ RODRIGUEZ,OLINDA BARRAGAN Y OTRA,,...,derecho al ambiente sano derecho a la salud m...,,1992-09-23,"[derecho, al, ambiente, sano, derecho, a, la, ...",[],"[derecho, ambiente, sano, derecho, salud, medi...",[],"(derecho, ambiente, sano, derecho, salud, medi...",(),1
7,8,3115,1992,9,24,http://www.corteconstitucional.gov.co//relator...,T-534/92,CIRO ANGARITA BARON,JORGE A MORENO,,...,derecho a la salud servicio militar derecho a...,,1992-09-24,"[derecho, a, la, salud, servicio, militar, der...",[],"[derecho, salud, servicio, militar, derecho, s...",[],"(derecho, salud, servicio, militar, derecho, s...",(),1
8,9,3479,1992,10,2,http://www.corteconstitucional.gov.co//relator...,T-548/92,CIRO ANGARITA BARON,FELIX ROJAS,,...,derecho a la salud derechos de los pacientes ...,,1992-10-02,"[derecho, a, la, salud, derechos, de, los, pac...",[],"[derecho, salud, derechos, pacientes, iss, neg...",[],"(derecho, salud, derechos, pacientes, iss, neg...",(),-1
9,10,D-039,1992,10,20,http://www.corteconstitucional.gov.co//relator...,C-560/92,SIMON RODRIGUEZ RODRIGUEZ,ABDON ORTIZ,,...,dec 127290 arts 3 lit p y 7 lit ll reforma de...,,1992-10-20,"[dec, 127290, arts, 3, lit, p, y, 7, lit, ll, ...",[],"[dec, 127290, arts, 3, lit, p, 7, lit, ll, ref...",[],"(dec, 127290, arts, 3, lit, p, 7, lit, ll, ref...",(),12
